In [4]:

import os.path as op

import numpy as np

import mne

#from library.config import (meg_dir, subjects_dir, N_JOBS, smooth,
#                            fsaverage_vertices, exclude_subjects, l_freq)

from scripts.processing.library.config import (meg_dir, subjects_dir, N_JOBS, smooth,
                            fsaverage_vertices, l_freq)

exclude_subjects = [1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17]
subject_EEG_layers = [2, 14, 18]

def morph_stc(subject_id):
    subject = "sub%03d" % subject_id
    print("processing subject: %s" % subject)
    data_path = op.join(meg_dir, subject)

    # Morph STCs
    morph_mat = None
    for condition in ('contrast', 'faces_eq', 'scrambled_eq'):
        stc = mne.read_source_estimate(
            op.join(data_path, 'mne_dSPM_inverse_highpass-%sHz-%s'
                    % (l_freq, condition)), subject)
        if morph_mat is None:
            morph_mat = mne.compute_morph_matrix(
                subject, 'fsaverage', stc.vertices, fsaverage_vertices, smooth,
                subjects_dir=subjects_dir, warn=False)
        morphed = stc.morph_precomputed('fsaverage', fsaverage_vertices,
                                        morph_mat)
        morphed.save(op.join(data_path,
                             'mne_dSPM_inverse_morph_highpass-%sHz-%s'
                             % (l_freq, condition)))
        if condition == 'contrast':
            out = morphed
    return out

In [3]:
for subject in range(0, len(subject_EEG_layers)):
    stcs = morph_stc(subject_EEG_layers[subject]) 

stcs = [stc for stc, subject_id in zip(stcs, range(1, 20))
        if subject_id not in exclude_subjects]
data = np.average([s.data for s in stcs], axis=0)
stc = mne.VectorSourceEstimate(data, stcs[0].vertices,
                               stcs[0].tmin, stcs[0].tstep, stcs[0].subject)
stc.save(op.join(meg_dir, 'contrast-average_highpass-%sHz' % l_freq))

processing subject: sub002


AttributeError: module 'mne' has no attribute 'compute_morph_matrix'